In [74]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import calendar

In [63]:
from sklearn import tree
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
import sys
import os

os.chdir(sys.path[3])

In [3]:
raw_data = pd.read_csv(r'data/process/new_data.csv', skiprows=[1, 2]).reset_index()

In [96]:
# Just for presentation
pd.read_csv(r'data/process/new_data.csv', skiprows=[2]).reset_index()

,index,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,...,Q33_9_TEXT,Q34,Q35,Q36,Q37,Q38,Q39,Q39_10_TEXT,Q40,Q40_3_TEXT
0,0,Start Date,End Date,Response Type,IP Address,Progress,Duration (in seconds),Finished,Recorded Date,Response ID,...,What were the main reasons you cycled more oft...,In what year were you born?,What is your gender?,What is your annual household income after taxes?,How many bicycles do you currently own?,What is the highest level of education you hav...,What do you consider your main ethnic origin o...,What do you consider your main ethnic origin o...,Where did you hear about this survey? - Select...,Where did you hear about this survey? - Other ...
1,1,4/21/2022 14:05,4/21/2022 14:10,IP Address,169.231.161.135,100,318,TRUE,4/21/2022 14:10,R_sRQ635z5e2hc1t7,...,NaN,1996,Woman,"$200,000 or more per year",One,"Graduate degree (e.g., Master’s, Professional ...","European (e.g., British Isles, German, French,...",NaN,Email,NaN
2,2,4/14/2022 17:28,4/14/2022 17:41,IP Address,128.111.106.41,22,770,FALSE,4/21/2022 17:41,R_21uo7CnrNd8VNIx,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,4/21/2022 19:27,4/21/2022 19:31,IP Address,70.77.83.154,100,229,TRUE,4/21/2022 19:31,R_7V5ufQv2WtbkiSR,...,NaN,1973,Man,I prefer to not answer,Three,Bachelor’s degree (university 4-year degree) ...,"European (e.g., British Isles, German, French,...",NaN,"A bicycle registry/recovery system (e.g., Bike...",NaN
4,4,4/14/2022 19:04,4/14/2022 20:18,IP Address,70.185.131.128,22,4454,FALSE,4/21/2022 20:18,R_qPdUVtdN6wMTtIt,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021,2021,6/6/2022 22:07,6/6/2022 22:14,IP Address,47.146.75.18,100,423,TRUE,6/6/2022 22:14,R_3fYRblhWSYPWCj8,...,NaN,1970,Woman,"Between $100,000 and $149,999 per year",Three,Bachelor’s degree (university 4-year degree) ...,"European (e.g., British Isles, German, French,...",NaN,Email,NaN
2022,2022,5/24/2022 12:17,5/24/2022 12:20,IP Address,174.90.223.242,69,191,FALSE,6/7/2022 12:20,R_3O7a8xS7l3i8EZG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023,2023,6/8/2022 3:19,6/8/2022 3:30,IP Address,108.211.181.111,100,697,TRUE,6/8/2022 3:30,R_3gO85Ni57rH0pTr,...,NaN,NaN,Prefer not to say,I prefer to not answer,I prefer to not answer,"Graduate degree (e.g., Master’s, Professional ...",I prefer to not answer,NaN,Other (please specify):,acquientance
2024,2024,5/25/2022 13:39,5/25/2022 13:39,IP Address,50.205.153.166,2,12,FALSE,6/8/2022 13:39,R_1jwTkRmNyNow9Tz,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



The questions we are going to use for the following model are stored in the cols variable. Q8 (What type of lock did you use to secure your bicycle? - Selected Choice) has been removed since it includes 951 Nun Values (that means 47% of participants didn't answer that question). The rest of the filtered questions have been removed since I thought they were irrelevant.

In [4]:
cols = ['index', 'Q2', 'Q4', 'Q5', 'Q6', 'Q7', 'Q9', 'Q10', 'Q11', 'Q12', 'Q13', 'Q14', 'Q15', 'Q17']

In [6]:
raw_features = raw_data[cols]

The code discards participants who answer more than three null questions

In [7]:
filtered_nun = raw_features[raw_features.count(axis=1) > 3]

In [51]:
for col in cols[1:]:
    print(filtered_nun.groupby(by=col, dropna=False).count()['index'])

Q2
A major bicycle part necessary for riding (i.e., saddle, wheel, pedal, or handlebar)      25
Entire bicycle (i.e., frame and components)                                             1826
NaN                                                                                       15
Name: index, dtype: int64
Q4
2017                    45
2018                    54
2019                   262
2020                   325
2021                   695
2022                   159
Don’t know/not sure      7
NaN                    319
Name: index, dtype: int64
Q5
April                  163
August                 145
December                85
Don’t know/not sure    131
February                88
January                 97
July                   137
June                   281
March                  116
May                    264
November                94
October                127
September              134
NaN                      4
Name: index, dtype: int64
Q6
Afternoon (12:00-17:59)    362
Don’t 

### Missing Values

First, a table is created, sorted from the question with the least NuNs to the question with the most NuNs

In [8]:
temp_dic = {'col_q': [], 'nun_count': []}
for col in cols[1:]:
    temp_groupby = filtered_nun.groupby(by=col, dropna=False).count()['index']
    temp_dic['col_q'].append(temp_groupby.index.name)
    temp_dic['nun_count'].append(temp_groupby[[np.nan]][0])
df_col_q_nun_count = DataFrame(temp_dic)
what_to_deal_first = df_col_q_nun_count.sort_values(by='nun_count')
what_to_deal_first

,col_q,nun_count
2,Q5,4
4,Q7,8
3,Q6,11
0,Q2,15
9,Q13,26
5,Q9,28
6,Q10,28
7,Q11,28
8,Q12,29
12,Q17,42


Encode the answers to the questions

In [81]:
encode_str = "encode_"
cols_encode = {col: encode_str + col for col in cols[1:]}

 Some columns should have manual encoding

In [86]:
manual_encoding = ['Q5', 'Q6', 'Q7', 'Q13']
q5_dic = {month: index for index, month in enumerate(calendar.month_name) if month}
q5_dic['Don’t know/not sure'] = 0
filtered_nun[encode_str + 'Q5'] = filtered_nun['Q5'].replace(q5_dic, regex=True).fillna(-1).astype(int)

{'January': 1,
 'February': 2,
 'March': 3,
 'April': 4,
 'May': 5,
 'June': 6,
 'July': 7,
 'August': 8,
 'September': 9,
 'October': 10,
 'November': 11,
 'December': 12,
 'Don’t know/not sure': 0}

In [103]:

q6_dic = {'Evening (18:00-23:59)': 3,
 'Morning (06:00-11:59)': 1,
 'Afternoon (12:00-17:59)': 2,
 'Overnight (00:00-05:59)': 4,
 'Don’t know/not sure': 0}
filtered_nun[encode_str + 'Q6'] = filtered_nun['Q6'].replace(q6_dic).fillna(-1).astype(int)
filtered_nun

C:\Users\Achituv\AppData\Local\Temp\ipykernel_13960\3537163205.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_nun[encode_str + 'Q6'] = filtered_nun['Q6'].replace(q6_dic).fillna(-1).astype(int)


,index,Q2,Q4,Q5,Q6,Q7,Q9,Q10,Q11,Q12,...,encode_Q6,encode_Q7,encode_Q9,encode_Q10,encode_Q11,encode_Q12,encode_Q13,encode_Q14,encode_Q15,encode_Q17
0,0,A major bicycle part necessary for riding (i.e...,2019,Don’t know/not sure,Evening (18:00-23:59),"Yes, frame locked to a fixed structure (e.g., ...",Outdoor bike rack,No,I did not report the theft,No,...,3,2,7,1,4,2,2,-1,-1,1
2,2,"Entire bicycle (i.e., frame and components)",2022,March,Morning (06:00-11:59),"Yes, frame locked to a fixed structure (e.g., ...",Other (please specify):,No,"The police ,A bicycle registry/recovery system...",No,...,1,2,6,1,10,2,0,2,0,0
3,3,"Entire bicycle (i.e., frame and components)",2017,May,Evening (18:00-23:59),"Yes, locked one or more wheels (i.e., bike loc...",NaN,NaN,NaN,NaN,...,3,3,-1,-1,-1,-1,-1,-1,-1,-1
4,4,"Entire bicycle (i.e., frame and components)",2022,March,Afternoon (12:00-17:59),"Yes, frame locked to a fixed structure (e.g., ...",Outdoor bike rack,Yes,The police,Yes,...,2,2,7,2,7,3,1,2,5,0
5,5,"Entire bicycle (i.e., frame and components)",2020,Don’t know/not sure,Evening (18:00-23:59),"No, unlocked",Outdoor bike rack,No,"The police ,A bicycle registry/recovery system...",No,...,3,1,7,1,8,2,4,2,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019,2019,"Entire bicycle (i.e., frame and components)",2022,March,Overnight (00:00-05:59),"No, unlocked","Outside in the yard or on a balcony, porch, or...",No,The police,Yes,...,4,1,8,1,7,3,2,2,8,1
2020,2020,"Entire bicycle (i.e., frame and components)",NaN,March,Morning (06:00-11:59),"No, unlocked",Inside a shed/garage,No,The police,No,...,1,1,4,1,7,2,0,2,8,0
2021,2021,"Entire bicycle (i.e., frame and components)",NaN,May,Evening (18:00-23:59),"No, unlocked",Inside a shed/garage,No,"The police ,A bicycle registry/recovery system...",No,...,3,1,4,1,10,2,1,2,8,0
2022,2022,"Entire bicycle (i.e., frame and components)",2017,Don’t know/not sure,Afternoon (12:00-17:59),"Yes, frame locked to a fixed structure (e.g., ...",Other (please specify):,No,The police,No,...,2,2,6,1,7,2,1,2,10,0


In [10]:
filtered_nun[list(cols_encode.values())] = filtered_nun[list(cols_encode.keys())].apply(
    lambda x: x.astype('category').cat.codes)
for col in cols_encode.items():
    col = list(col)
    print(filtered_nun[col].apply(lambda x: x.drop_duplicates()))

                                                   Q2  encode_Q2
0   A major bicycle part necessary for riding (i.e...          0
2        Entire bicycle (i.e., frame and components)           1
38                                                NaN         -1
                      Q4  encode_Q4
0                   2019          2
2                   2022          5
3                   2017          0
5                   2020          3
6                   2021          4
7                    NaN         -1
55                  2018          1
411  Don’t know/not sure          6
                     Q5  encode_Q5
0   Don’t know/not sure          3
2                March           8
3                  May           9
6                April           0
16            December           2
23            November          10
29                July           6
35             October          11
37            February           4
39              August           1
43                June         

C:\Users\Achituv\AppData\Local\Temp\ipykernel_13960\309705082.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_nun[list(cols_encode.values())]  = filtered_nun[list(cols_encode.keys())].apply(lambda x: x.astype('category').cat.codes)
C:\Users\Achituv\AppData\Local\Temp\ipykernel_13960\309705082.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_nun[list(cols_encode.values())]  = filtered_nun[list(cols_encode.keys())].apply(lambda x: x.astype('category').cat.codes)
C:\Users\Achituv\AppDa

# Sol 1

The NuNs of the question with the fewest NuNs will be replaced with the most popular answer

In [17]:
sol1 = filtered_nun.copy()
min_nun = cols_encode[what_to_deal_first.iloc[0]['col_q']]
print("The question with the fewest NuN values is {} ".format(min_nun))
replace_nun = filtered_nun.groupby(by=min_nun, dropna=False).count()['index'].idxmax()
print("the most popular answer for {} is {} ".format(min_nun, replace_nun))
sol1[min_nun] = sol1[min_nun].replace(-1, replace_nun)
sol1

The question with the fewest NuN values is encode_Q5 
the most popular answer for encode_Q5 is 7 


,index,Q2,Q4,Q5,Q6,Q7,Q9,Q10,Q11,Q12,...,encode_Q6,encode_Q7,encode_Q9,encode_Q10,encode_Q11,encode_Q12,encode_Q13,encode_Q14,encode_Q15,encode_Q17
0,0,A major bicycle part necessary for riding (i.e...,2019,Don’t know/not sure,Evening (18:00-23:59),"Yes, frame locked to a fixed structure (e.g., ...",Outdoor bike rack,No,I did not report the theft,No,...,2,2,7,1,4,2,2,-1,-1,1
2,2,"Entire bicycle (i.e., frame and components)",2022,March,Morning (06:00-11:59),"Yes, frame locked to a fixed structure (e.g., ...",Other (please specify):,No,"The police ,A bicycle registry/recovery system...",No,...,3,2,6,1,10,2,0,2,0,0
3,3,"Entire bicycle (i.e., frame and components)",2017,May,Evening (18:00-23:59),"Yes, locked one or more wheels (i.e., bike loc...",NaN,NaN,NaN,NaN,...,2,3,-1,-1,-1,-1,-1,-1,-1,-1
4,4,"Entire bicycle (i.e., frame and components)",2022,March,Afternoon (12:00-17:59),"Yes, frame locked to a fixed structure (e.g., ...",Outdoor bike rack,Yes,The police,Yes,...,0,2,7,2,7,3,1,2,5,0
5,5,"Entire bicycle (i.e., frame and components)",2020,Don’t know/not sure,Evening (18:00-23:59),"No, unlocked",Outdoor bike rack,No,"The police ,A bicycle registry/recovery system...",No,...,2,1,7,1,8,2,4,2,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019,2019,"Entire bicycle (i.e., frame and components)",2022,March,Overnight (00:00-05:59),"No, unlocked","Outside in the yard or on a balcony, porch, or...",No,The police,Yes,...,4,1,8,1,7,3,2,2,8,1
2020,2020,"Entire bicycle (i.e., frame and components)",NaN,March,Morning (06:00-11:59),"No, unlocked",Inside a shed/garage,No,The police,No,...,3,1,4,1,7,2,0,2,8,0
2021,2021,"Entire bicycle (i.e., frame and components)",NaN,May,Evening (18:00-23:59),"No, unlocked",Inside a shed/garage,No,"The police ,A bicycle registry/recovery system...",No,...,2,1,4,1,10,2,1,2,8,0
2022,2022,"Entire bicycle (i.e., frame and components)",2017,Don’t know/not sure,Afternoon (12:00-17:59),"Yes, frame locked to a fixed structure (e.g., ...",Other (please specify):,No,The police,No,...,0,2,6,1,7,2,1,2,10,0


By using a random forest tree, impute the missing values in the rest of the columns

In [19]:
clf = RandomForestClassifier(random_state=0)
x_columns = [min_nun]
for col in what_to_deal_first['col_q'][1:]:
    cur_field = cols_encode[col]
    print(col)
    x_columns.append(cur_field)
    data = sol1.loc[filtered_nun[cur_field] > -1, x_columns]
    X = data[x_columns[:-1]].to_numpy()
    y = data[x_columns[-1]].to_numpy()
    clf.fit(X, y)
    print(clf.score(X, y))
    data_to_predict = sol1.loc[filtered_nun[cur_field] == -1, x_columns[:-1]].to_numpy()
    sol1.loc[filtered_nun[cur_field] == -1, cur_field] = clf.predict(data_to_predict)

Q7
0.5220667384284177
Q6
0.4123989218328841
Q2
0.9870340356564019
Q13
0.3858695652173913
Q9
0.6387377584330794
Q10
0.8781284004352558
Q11
0.8824809575625681
Q12
0.9695155144256941
Q17
0.993421052631579
Q15
0.9766407119021134
Q14
1.0
Q4
0.9903038138332256


## Sol 2

In [26]:
sol2 = filtered_nun.copy()
clf = RandomForestClassifier(random_state=0)
for col in what_to_deal_first['col_q']:
    print(col)
    y_col = cols_encode[col]
    # Get only encode columns from the table
    pre_filter = sol2[list(cols_encode.values())]

    # get only rows with Nun (-1) in current cul
    pre_x = pre_filter[pre_filter[y_col] == -1].drop(columns=y_col)
    # Find columns with Nun values. That columns would not consider features
    res = pre_x[pre_x == -1].any()
    no_features_list = list(res[res == True].index)
    print('number of features are:{}'.format(len(cols) - 2 - len(no_features_list)))
    x_predict = pre_x.drop(columns=no_features_list)
    features_labels = pre_filter.drop(columns=no_features_list)

    # Build the data for the prediction
    xy = features_labels[~features_labels[features_labels == -1].any(axis=1)]
    x = xy.drop(columns=y_col).to_numpy()
    y = xy[y_col].to_numpy()

    clf.fit(x, y)
    print(clf.score(x, y))
    sol2.loc[sol2[y_col] == -1, y_col] = clf.predict(x_predict)
sol2

Q5
number of features are:10
0.8723041997729852


C:\Users\Achituv\anaconda3\envs\ox\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
C:\Users\Achituv\anaconda3\envs\ox\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
C:\Users\Achituv\anaconda3\envs\ox\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


Q7
number of features are:3
0.5547540983606557
Q6
number of features are:3
0.41630434782608694
Q2
number of features are:9
1.0
Q13
number of features are:4
0.3864130434782609
Q9
number of features are:5


C:\Users\Achituv\anaconda3\envs\ox\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
C:\Users\Achituv\anaconda3\envs\ox\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


0.6376496191512514
Q10
number of features are:6
0.8775843307943417
Q11
number of features are:7


...

0.993421052631579
Q15
number of features are:10
0.9766407119021134
Q14
number of features are:11


C:\Users\Achituv\anaconda3\envs\ox\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
C:\Users\Achituv\anaconda3\envs\ox\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


1.0
Q4
number of features are:12
0.9883645765998708


C:\Users\Achituv\anaconda3\envs\ox\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
C:\Users\Achituv\anaconda3\envs\ox\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


,index,Q2,Q4,Q5,Q6,Q7,Q9,Q10,Q11,Q12,...,encode_Q6,encode_Q7,encode_Q9,encode_Q10,encode_Q11,encode_Q12,encode_Q13,encode_Q14,encode_Q15,encode_Q17
0,0,A major bicycle part necessary for riding (i.e...,2019,Don’t know/not sure,Evening (18:00-23:59),"Yes, frame locked to a fixed structure (e.g., ...",Outdoor bike rack,No,I did not report the theft,No,...,2,2,7,1,4,2,2,2,10,1
2,2,"Entire bicycle (i.e., frame and components)",2022,March,Morning (06:00-11:59),"Yes, frame locked to a fixed structure (e.g., ...",Other (please specify):,No,"The police ,A bicycle registry/recovery system...",No,...,3,2,6,1,10,2,0,2,0,0
3,3,"Entire bicycle (i.e., frame and components)",2017,May,Evening (18:00-23:59),"Yes, locked one or more wheels (i.e., bike loc...",NaN,NaN,NaN,NaN,...,2,3,9,1,9,3,8,2,1,0
4,4,"Entire bicycle (i.e., frame and components)",2022,March,Afternoon (12:00-17:59),"Yes, frame locked to a fixed structure (e.g., ...",Outdoor bike rack,Yes,The police,Yes,...,0,2,7,2,7,3,1,2,5,0
5,5,"Entire bicycle (i.e., frame and components)",2020,Don’t know/not sure,Evening (18:00-23:59),"No, unlocked",Outdoor bike rack,No,"The police ,A bicycle registry/recovery system...",No,...,2,1,7,1,8,2,4,2,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019,2019,"Entire bicycle (i.e., frame and components)",2022,March,Overnight (00:00-05:59),"No, unlocked","Outside in the yard or on a balcony, porch, or...",No,The police,Yes,...,4,1,8,1,7,3,2,2,8,1
2020,2020,"Entire bicycle (i.e., frame and components)",NaN,March,Morning (06:00-11:59),"No, unlocked",Inside a shed/garage,No,The police,No,...,3,1,4,1,7,2,0,2,8,0
2021,2021,"Entire bicycle (i.e., frame and components)",NaN,May,Evening (18:00-23:59),"No, unlocked",Inside a shed/garage,No,"The police ,A bicycle registry/recovery system...",No,...,2,1,4,1,10,2,1,2,8,0
2022,2022,"Entire bicycle (i.e., frame and components)",2017,Don’t know/not sure,Afternoon (12:00-17:59),"Yes, frame locked to a fixed structure (e.g., ...",Other (please specify):,No,The police,No,...,0,2,6,1,7,2,1,2,10,0


## Decision Tree for Recovery

In [ ]:
recovery_col = 'Q18'
recovery_col_encode = "encode_" + recovery_col
sol2[recovery_col] = raw_data[recovery_col]

In [33]:
print(sol2.groupby(by=recovery_col, dropna=False).count()['index'])

Q18
No                             1555
Yes (including if damaged)      281
NaN                              30
Name: index, dtype: int64


In [58]:
# Rows without recovery information should be discarded
pre_dt = sol2[sol2[recovery_col].notna()]
# Encode recovery (0 or 1)
pre_dt[recovery_col_encode] = pre_dt[recovery_col].astype('category').cat.codes
pre_dt

C:\Users\Achituv\AppData\Local\Temp\ipykernel_13960\1900047526.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pre_dt[recovery_col_encode] = pre_dt[recovery_col].astype('category').cat.codes


,index,Q2,Q4,Q5,Q6,Q7,Q9,Q10,Q11,Q12,...,encode_Q9,encode_Q10,encode_Q11,encode_Q12,encode_Q13,encode_Q14,encode_Q15,encode_Q17,Q18,encode_Q18
0,0,A major bicycle part necessary for riding (i.e...,2019,Don’t know/not sure,Evening (18:00-23:59),"Yes, frame locked to a fixed structure (e.g., ...",Outdoor bike rack,No,I did not report the theft,No,...,7,1,4,2,2,2,10,1,No,0
2,2,"Entire bicycle (i.e., frame and components)",2022,March,Morning (06:00-11:59),"Yes, frame locked to a fixed structure (e.g., ...",Other (please specify):,No,"The police ,A bicycle registry/recovery system...",No,...,6,1,10,2,0,2,0,0,No,0
4,4,"Entire bicycle (i.e., frame and components)",2022,March,Afternoon (12:00-17:59),"Yes, frame locked to a fixed structure (e.g., ...",Outdoor bike rack,Yes,The police,Yes,...,7,2,7,3,1,2,5,0,Yes (including if damaged),1
5,5,"Entire bicycle (i.e., frame and components)",2020,Don’t know/not sure,Evening (18:00-23:59),"No, unlocked",Outdoor bike rack,No,"The police ,A bicycle registry/recovery system...",No,...,7,1,8,2,4,2,8,1,No,0
6,6,"Entire bicycle (i.e., frame and components)",2021,April,Overnight (00:00-05:59),"No, unlocked",Inside a shed/garage,No,"The police ,A bicycle registry/recovery system...",Yes,...,4,1,10,3,3,2,8,0,No,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019,2019,"Entire bicycle (i.e., frame and components)",2022,March,Overnight (00:00-05:59),"No, unlocked","Outside in the yard or on a balcony, porch, or...",No,The police,Yes,...,8,1,7,3,2,2,8,1,No,0
2020,2020,"Entire bicycle (i.e., frame and components)",NaN,March,Morning (06:00-11:59),"No, unlocked",Inside a shed/garage,No,The police,No,...,4,1,7,2,0,2,8,0,No,0
2021,2021,"Entire bicycle (i.e., frame and components)",NaN,May,Evening (18:00-23:59),"No, unlocked",Inside a shed/garage,No,"The police ,A bicycle registry/recovery system...",No,...,4,1,10,2,1,2,8,0,No,0
2022,2022,"Entire bicycle (i.e., frame and components)",2017,Don’t know/not sure,Afternoon (12:00-17:59),"Yes, frame locked to a fixed structure (e.g., ...",Other (please specify):,No,The police,No,...,6,1,7,2,1,2,10,0,No,0


In [ ]:


clf = DecisionTreeClassifier(random_state=0)

In [57]:
x_tree = pre_dt[list(cols_encode.values())].to_numpy()
y_tree = pre_dt[recovery_col_encode].to_numpy()


array([0, 0, 1, ..., 0, 0, 0], dtype=int8)

In [59]:
clf.fit(x_tree, y_tree)
print(clf.score(x_tree, y_tree))

0.9983660130718954


In [60]:
clf.feature_importances_

array([0.        , 0.08831449, 0.17808391, 0.09184536, 0.02586858,
       0.133657  , 0.03973051, 0.10534414, 0.05520821, 0.12107651,
       0.02833158, 0.11535265, 0.01718707])

In [70]:
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(20, 20), dpi=500)
tree.plot_tree(clf,
               feature_names=cols[1:],
               max_depth=4,
               class_names=['not_recovered', 'recovered'],
               filled=True)
fig.savefig('imagename.pdf')

# general analysis on recovery